In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

In [3]:
xlsx = pd.ExcelFile('data_telco.xlsx')
telco = pd.read_excel(xlsx, 'data_telco')

In [4]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 42 columns):
region                          1000 non-null object
Months with service             1000 non-null int64
age                             1000 non-null int64
Marital Status                  1000 non-null int64
Years at current address        1000 non-null int64
Household income in thousand    1000 non-null int64
Level of Education              1000 non-null int64
Years with current employer     1000 non-null int64
Retired                         1000 non-null int64
Gender                          1000 non-null object
No. of people in household      1000 non-null int64
Tollfree service                1000 non-null int64
Equipment rental                1000 non-null int64
Calling card service            1000 non-null int64
Wireless Service                1000 non-null int64
Long distance last month        1000 non-null float64
Tollfree last month             1000 non-null float64
Equi

In [5]:
total = telco.isnull().sum()
percent1 = telco.isnull().sum()*100/telco.isnull().count()
percent = (round(percent1,1))
missing_data = pd.concat([total,percent],axis = 1,keys = ['total_null','percent_null']).transpose()
missing_data

,region,Months with service,age,Marital Status,Years at current address,Household income in thousand,Level of Education,Years with current employer,Retired,Gender,...,3-way calling,Electronic billing,Log-long distance,logtoll,Log-equipment,Log-calling card,Log-wireless,Log-income,Customer category,Churn within last month
total_null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,525.0,614.0,322.0,704.0,0.0,0.0,0.0
percent_null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,52.5,61.4,32.2,70.4,0.0,0.0,0.0


In [6]:
def deep_describe(data):
    cmsq = data.describe()
    
    mode = data.mode(axis = 0,numeric_only=True)
    mode = mode.dropna(axis = 0,how='any')
    mode = mode.set_index([['mode']])
    
    median = pd.DataFrame(data.median()).transpose()
    median = median.set_index([['median']])
    
    kurtosis = pd.DataFrame(data.kurtosis()).transpose()
    kurtosis = kurtosis.set_index([['kurtosis']])
    
    skew = pd.DataFrame(data.skew()).transpose()
    skew = skew.set_index([['skewness']])
    
    tr = [cmsq,mode,median,skew,kurtosis]
    return pd.concat(tr)
    
deep_describe(telco)

,Months with service,age,Marital Status,Years at current address,Household income in thousand,Level of Education,Years with current employer,Retired,No. of people in household,Tollfree service,...,Call forwarding,3-way calling,Electronic billing,Log-long distance,logtoll,Log-equipment,Log-calling card,Log-wireless,Log-income,Churn within last month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,475.000000,386.000000,678.000000,296.000000,1000.000000,1000.000000
mean,35.526000,41.684000,0.495000,11.551000,77.535000,2.671000,10.987000,0.047000,2.331000,0.474000,...,0.493000,0.502000,0.371000,2.181930,3.239874,3.568135,2.854027,3.597872,3.957000,0.274000
std,21.359812,12.558816,0.500225,10.086681,107.044165,1.222397,10.082087,0.211745,1.435793,0.499573,...,0.500201,0.500246,0.483314,0.734301,0.413795,0.277567,0.557612,0.367397,0.803548,0.446232
min,1.000000,18.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,-0.110000,1.750000,2.730000,1.010000,2.700000,2.200000,0.000000
25%,17.000000,32.000000,0.000000,3.000000,29.000000,2.000000,3.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.650000,2.970000,3.370000,2.460000,3.330000,3.370000,0.000000
50%,34.000000,40.000000,0.000000,9.000000,47.000000,3.000000,8.000000,0.000000,2.000000,0.000000,...,0.000000,1.000000,0.000000,2.145000,3.210000,3.570000,2.850000,3.590000,3.850000,0.000000
75%,54.000000,51.000000,1.000000,18.000000,83.000000,4.000000,17.000000,0.000000,3.000000,1.000000,...,1.000000,1.000000,1.000000,2.670000,3.490000,3.760000,3.210000,3.860000,4.420000,1.000000
max,72.000000,77.000000,1.000000,55.000000,1668.000000,5.000000,47.000000,1.000000,8.000000,1.000000,...,1.000000,1.000000,1.000000,4.600000,5.150000,4.350000,4.690000,4.720000,7.420000,1.000000
mode,72.000000,33.000000,0.000000,1.000000,25.000000,2.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.450000,3.310000,3.670000,2.930000,3.330000,3.220000,0.000000
median,34.000000,40.000000,0.000000,9.000000,47.000000,3.000000,8.000000,0.000000,2.000000,0.000000,...,0.000000,1.000000,0.000000,2.145000,3.210000,3.570000,2.850000,3.590000,3.850000,0.000000


Now we have the 4 quartiles, count of the values, standard deviation and min and max values of data with all mean mode and median with skewness and kurtosis of data.

Now we make the correlation matrix so that we can select the data which we should include in our regression tree.
Now since the data from 4 columns is more than 30% missing so we drop those columns.

We have catagorical variables so we just use dummy fuction of pandas.

In [7]:
tel_data = telco.dropna(axis = 1)
tel_data = pd.get_dummies(tel_data)

In [8]:
targets = pd.DataFrame(telco['Churn within last month'])

features = tel_data.drop(['Churn within last month'],axis = 1)
features.info()
x_train,x_test,y_train,y_test = train_test_split(features,targets,train_size = 0.7,random_state = 42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 43 columns):
Months with service                1000 non-null int64
age                                1000 non-null int64
Marital Status                     1000 non-null int64
Years at current address           1000 non-null int64
Household income in thousand       1000 non-null int64
Level of Education                 1000 non-null int64
Years with current employer        1000 non-null int64
Retired                            1000 non-null int64
No. of people in household         1000 non-null int64
Tollfree service                   1000 non-null int64
Equipment rental                   1000 non-null int64
Calling card service               1000 non-null int64
Wireless Service                   1000 non-null int64
Long distance last month           1000 non-null float64
Tollfree last month                1000 non-null float64
Equipment last month               1000 non-null float64
Calling 

c:\users\aish.gupta\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
logit = LogisticRegression()
fitted_logit = logit.fit(x_train,y_train)

c:\users\aish.gupta\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
test_telco = fitted_logit.predict(x_test)

In [11]:
pd.DataFrame(confusion_matrix(y_test,test_telco))
roc_auc_score(y_test,test_telco)

0.681296340588376

In [21]:
from sklearn.linear_model import LogisticRegressionCV

tel_data_CV = telco.dropna(axis = 1)

features_CV = tel_data.drop(['Churn within last month'],axis = 1)

x_train_CV,x_test_CV,y_train_CV,y_test_CV = train_test_split(features,targets,train_size = 0.7,random_state = 42)

logit_CV = LogisticRegressionCV(cv=5)
fitted_logit_CV = logit_CV.fit(x_train_CV,y_train_CV)

test_telco_CV = fitted_logit_CV.predict(x_test_CV)

pd.DataFrame(confusion_matrix(y_test_CV,test_telco_CV))
roc_auc_score(y_test_CV,test_telco_CV)

c:\users\aish.gupta\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
c:\users\aish.gupta\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6539105477158574

In [52]:
prob = pd.DataFrame(fitted_logit_CV.predict_proba(x_test_CV))
given = y_test_CV.reset_index().drop(['index'],axis = 1)

given.join(prob)

,Churn within last month,0,1
0,0,0.909952,0.090048
1,1,0.666937,0.333063
2,0,0.580444,0.419556
3,0,0.889714,0.110286
4,1,0.555770,0.444230
5,0,0.703022,0.296978
6,1,0.392743,0.607257
7,0,0.685163,0.314837
8,1,0.617198,0.382802
9,0,0.791972,0.208028
